In [1]:
!python -c "import monai; import nibabel; import tqdm" || pip install -q "monai-weekly[nibabel, tqdm]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

21번과 비교실험 

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import shutil
import tempfile
import nibabel as nib
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.transforms import (
    AsDiscrete,
    Compose,
    Resized,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandSpatialCropSamplesd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    EnsureTyped,
    RandGaussianNoised,
    RandGaussianSmoothd,
    RandZoomd,
    RandFlip,
    RandRotate90,
    RandAdjustContrast,
    RandShiftIntensity,
    RandSimulateLowResolutiond
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import (SwinUNETR, UNETR, UNet, DynUNet, SegResNet)

from monai.data import (
    ThreadDataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
    set_track_meta,
)

import torch
from torch import nn
from torch.optim.lr_scheduler import _LRScheduler
def get_kernels_strides(patch_size, spacing):
    """
    This function is only used for decathlon datasets with the provided patch sizes.
    When refering this method for other tasks, please ensure that the patch size for each spatial dimension should
    be divisible by the product of all strides in the corresponding dimension.
    In addition, the minimal spatial size should have at least one dimension that has twice the size of
    the product of all strides. For patch sizes that cannot find suitable strides, an error will be raised.

    """
    sizes, spacings = patch_size, spacing
    input_size = sizes
    strides, kernels = [], []
    while True:
        spacing_ratio = [sp / min(spacings) for sp in spacings]
        stride = [2 if ratio <= 2 and size >= 8 else 1 for (ratio, size) in zip(spacing_ratio, sizes)]
        kernel = [3 if ratio <= 2 else 1 for ratio in spacing_ratio]
        if all(s == 1 for s in stride):
            break
        for idx, (i, j) in enumerate(zip(sizes, stride)):
            if i % j != 0:
                raise ValueError(
                    f"Patch size is not supported, please try to modify the size {input_size[idx]} in the spatial dimension {idx}."
                )
        sizes = [i / j for i, j in zip(sizes, stride)]
        spacings = [i * j for i, j in zip(spacings, stride)]
        kernels.append(kernel)
        strides.append(stride)

    strides.insert(0, len(spacings) * [1])
    kernels.append(len(spacings) * [3])
    return kernels, strides
class PolyLRScheduler(_LRScheduler):
    def __init__(self, optimizer, initial_lr: float, max_steps: int, exponent: float = 0.9, current_step: int = None):
        self.optimizer = optimizer
        self.initial_lr = initial_lr
        self.max_steps = max_steps
        self.exponent = exponent
        self.ctr = 0
        super().__init__(optimizer, current_step if current_step is not None else -1, False)

    def step(self, current_step=None):
        if current_step is None or current_step == -1:
            current_step = self.ctr
            self.ctr += 1

        new_lr = self.initial_lr * (1 - current_step / self.max_steps) ** self.exponent
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = new_lr

In [3]:
def oar_run_train(json_name):
    
    root_dir = r"D:\! project\2024- AortaSeg\data\new_final\zone3"
    data_dir = root_dir + r"/"
    num_samples = 2
    model_savepath = os.path.join(root_dir, "20240926model_%s" %json_name)
    loss_savepath = os.path.join(root_dir, "20240926loss_%s" %json_name)
    os.makedirs(model_savepath)
    os.makedirs(loss_savepath)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    space_x, space_y, space_z = 1.1, 1.1, 2.0
    a_min, a_max, b_min, b_max = -175, 350, 0, 1
    spatial_size_xyz = (128, 112, 160)
    
    train_transforms = Compose(
            [
                LoadImaged(keys=["image", "label"], ensure_channel_first=True),
                # Orientationd(keys=["image", "label"], axcodes="LPS"),
                EnsureTyped(keys=["image", "label"], device=device, track_meta=False),
                RandRotate90d(
                     keys=["image", "label"],
                     prob=0.4,
                     max_k=3,
                     spatial_axes=(0, 1)
                ),
                RandRotate90d(
                     keys=["image", "label"],
                     prob=0.3,
                     max_k=3,
                     spatial_axes=(1, 2)
                ),
                RandRotate90d(
                     keys=["image", "label"],
                     prob=0.3,
                     max_k=3,
                     spatial_axes=(2, 0)
                ),
                RandZoomd(
                    keys=["image", "label"],
                    min_zoom=0.7,
                    max_zoom=1.4,
                    mode=("trilinear", "nearest"),
                    align_corners=(True, None),
                    prob=0.20,
                ),
                # RandFlipd(["image", "label"], spatial_axis=[0], prob=0.1),
                # RandFlipd(["image", "label"], spatial_axis=[1], prob=0.1),
                # RandFlipd(["image", "label"], spatial_axis=[2], prob=0.1),
                RandCropByPosNegLabeld(
                    keys=["image", "label"],
                    label_key="label",
                    spatial_size=spatial_size_xyz,
                    pos=1,
                    neg=1,
                    num_samples=num_samples,
                    image_key="image",
                    image_threshold=0,
                ),
                # RandSpatialCropSamplesd(
                #     keys=["image", "label"],
                #     roi_size=spatial_size_xyz,
                #     num_samples=num_samples,
                #     random_size=False
                # ),
                RandSimulateLowResolutiond(keys=["image"], prob=0.3),
                RandGaussianNoised(keys=["image"], std=0.01, prob=0.1),
                RandGaussianSmoothd(
                    keys=["image"],
                    sigma_x=(0.5, 1.15),
                    sigma_y=(0.5, 1.15),
                    sigma_z=(0.5, 1.15),
                    prob=0.2    
                ),
                
                # RandRotate90d(
                #      keys=["image", "label"],
                #      prob=0.3,
                #      max_k=3,
                # )
                # RandShiftIntensityd(
                #     keys=["image"],
                #     offsets=0.10,
                #     prob=0.50,
                # ),
            ]
            ) 
    
        
    
    val_transforms = Compose(
        [
            LoadImaged(keys=["image", "label"], ensure_channel_first=True),
            # ScaleIntensityRanged(keys=["image"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            # Orientationd(keys=["image", "label"], axcodes="LPS"),
            EnsureTyped(keys=["image", "label"], device=device, track_meta=True),
        ]
    )

    test_transforms = Compose(
        [
            LoadImaged(keys=["image", "label"], ensure_channel_first=True),
            ScaleIntensityRanged(keys=["image"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            Orientationd(keys=["image", "label"], axcodes="LPS"),
            Spacingd(
                keys=["image", "label"],
                pixdim=(space_x, space_y, space_z),
                mode=("bilinear", "nearest"),
            ),
            EnsureTyped(keys=["image", "label"], device=device, track_meta=True),
        ]
    )

    split_json = json_name 
    
    datasets = data_dir + split_json
    datalist = load_decathlon_datalist(datasets, True, "training")
    val_files = load_decathlon_datalist(datasets, True, "validation")
    train_ds = CacheDataset(
        data=datalist,
        transform=train_transforms,
        cache_num=40,
        cache_rate=1.0,
        num_workers=8,
    )
    train_loader = ThreadDataLoader(train_ds, num_workers=0, batch_size=1, shuffle=True)
    val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_num=0, cache_rate=1.0, num_workers=4)
    val_loader = ThreadDataLoader(val_ds, num_workers=0, batch_size=1)


    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    OAR_nums_plus_one = 9
    patch_size = list(spatial_size_xyz)
    spacing = [1.0, 1.0, 1.0]
    ks, st = get_kernels_strides(patch_size, spacing)
    print(ks, st)
    uks = st[1:]
    #dynunet hyperparameter
   
    # ks = [[3, 3, 1], [3, 3, 3], [3, 3, 3], [3, 3, 3]]
    # st = [[1, 1, 1], [2, 2, 1], [2, 2, 2], [2, 2, 1]]
    uks = st[1:]

    # model = UNETR(
    # img_size=spatial_size_xyz,
    # in_channels=5,
    # out_channels=OAR_nums_plus_one,
    # ).to(device)
    # model = SegResNet(in_channels = 5, out_channels = OAR_nums_plus_one, dropout_prob = 0.3, act="LEAKYRELU").to(device)
    model = DynUNet(
        spatial_dims=3,
        in_channels=5,
        out_channels=OAR_nums_plus_one,
        kernel_size=ks,
        strides=st,
        upsample_kernel_size=uks,
        dropout=0.1,
        act_name= "LEAKYRELU",
        deep_supervision=False
    ).to(device)
    # model = SwinUNETR(
    # img_size=(64, 64, 32),
    # in_channels=5,
    # out_channels=OAR_nums_plus_one,
    # feature_size=48,
    # use_checkpoint=False,
    # ).to(device)

    torch.backends.cudnn.benchmark = True
    #FOR SINGLE CHANNEL PREDICTION
    loss_function = DiceCELoss(to_onehot_y=OAR_nums_plus_one, softmax=True)
    initial_lr = 1e-2
    optimizer = torch.optim.SGD(model.parameters(), lr=initial_lr, weight_decay=3e-5, momentum=0.99, nesterov=True)
    # scheduler = CosineAnnealingNoWarmUpRestarts(optimizer, 5, 2, 1e-2, 0, 0, 0.8)
    T_0 = 20
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=T_0, T_mult=2, eta_min=0)
    scheduler = PolyLRScheduler(optimizer, initial_lr=initial_lr, max_steps=2000)
    scaler = torch.cuda.amp.GradScaler()


    import gc
    import random
    def validation(epoch_iterator_val):
        model.eval()
        with torch.no_grad():
            total_min = -1024
            total_max = 3071
            bone_min = -1000
            bone_max = 2000
            soft_min = -160
            soft_max = 350
            brain_min = -5
            brain_max = 65
            stroke_min = 15
            stroke_max = 45

            for batch in epoch_iterator_val:
                gc.collect()
                torch.cuda.empty_cache()
                val_inputs, val_labels = (batch["image"].cuda(), batch["label"].cuda())
                
                tx =  torch.reshape(val_inputs[:, 0, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
                x1 =  torch.reshape(val_inputs[:, 0, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
                x2 =  torch.reshape(val_inputs[:, 0, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
                x3 =  torch.reshape(val_inputs[:, 0, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
                x4 =  torch.reshape(val_inputs[:, 0, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
                tx[tx<total_min] = total_min
                tx[tx>total_max] = total_max
                tx = (tx-total_min)/(total_max-total_min)
                x1[x1<bone_min] = bone_min
                x1[x1>bone_max] = bone_max
                x1 = (x1-bone_min)/(bone_max-bone_min)
                x2[x2<soft_min] = soft_min
                x2[x2>soft_max] = soft_max
                x2 = (x2-soft_min)/(soft_max-soft_min)
                x3[x3<brain_min] = brain_min
                x3[x3>brain_max] = brain_max 
                x3 = (x3-brain_min)/(brain_max-brain_min)
                x4[x4<stroke_min] = stroke_min
                x4[x4>stroke_max] = stroke_max
                x4 = (x4 - stroke_min)/(stroke_max - stroke_min)
                
                tx *= 2
                x2 *= 2
                tx -= 1
                x2 -= 1

                val_inputs = torch.cat((tx, x1, x2, x3, x4), 1)

                with torch.cuda.amp.autocast():
                    val_outputs = sliding_window_inference(val_inputs, spatial_size_xyz, 4, model)
                val_labels_list = decollate_batch(val_labels)
                val_labels_convert = [post_label(val_label_tensor) for val_label_tensor in val_labels_list]
                val_outputs_list = decollate_batch(val_outputs)
                val_output_convert = [post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list]
                dice_metric(y_pred=val_output_convert, y=val_labels_convert)
                epoch_iterator_val.set_description("Validate (%d / %d Steps)" % (global_step, 10.0))
            mean_dice_val = dice_metric.aggregate().item()
            dice_metric.reset()
        return mean_dice_val

    from PIL import Image
    def train(global_step, train_loader, dice_val_best, global_step_best, t_0):
        model.train()
        epoch_loss = 0
        step = 0
        # rand_fliper_x = RandFlip(prob=0.5, spatial_axis=0)
        # rand_fliper_y = RandFlip(prob=0.1, spatial_axis=1)
        # rand_fliper_z = RandFlip(prob=0.1, spatial_axis=2)

        epoch_iterator = tqdm(train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True)
        print(optimizer.param_groups[0]['lr'])
        randcontrast = RandAdjustContrast(prob=0.3, gamma=(0.7, 1.5))
        randintensity = RandShiftIntensity(prob=0.15, offsets=0.4)
        for step, batch in enumerate(epoch_iterator):
            step += 1
            x, y = (batch["image"].cuda(), batch["label"].cuda())
            # img = x[0, 0, ...].clone.detach().cpu().numpy()
            
            #print(x.shape)
            #x = torch.transpose(x, 1, 0)
           # print(x.shape)
           # x = rand_fliper_x(x)
          #  x = rand_fliper_y(x)
           # x = rand_fliper_z(x)
           # x = torch.transpose(x, 1, 0)
          #  print(x.shape)
            total_min = -1024
            total_max = 3071
            bone_min = random.randrange(-1024, -699, 1)
            bone_max = random.randrange(1850, 2151, 1)
            soft_min = random.randrange(-250, -149, 1)
            soft_max = random.randrange(290, 391, 1)
            brain_min = random.randrange(-10, 0, 1)
            brain_max = random.randrange(40, 91, 1)
            stroke_min = random.randrange(10, 21, 1)
            stroke_max = random.randrange(40, 51, 1)
            
            tx =  torch.reshape(x[:, 0, :, : ,:].clone().detach(), (x.shape[0], 1, x.shape[2], x.shape[3], x.shape[4]))
            x1 =  torch.reshape(x[:, 0, :, : ,:].clone().detach(), (x.shape[0], 1, x.shape[2], x.shape[3], x.shape[4]))
            x2 =  torch.reshape(x[:, 0, :, : ,:].clone().detach(), (x.shape[0], 1, x.shape[2], x.shape[3], x.shape[4]))
            x3 =  torch.reshape(x[:, 0, :, : ,:].clone().detach(), (x.shape[0], 1, x.shape[2], x.shape[3], x.shape[4]))
            x4 =  torch.reshape(x[:, 0, :, : ,:].clone().detach(), (x.shape[0], 1, x.shape[2], x.shape[3], x.shape[4]))
            
            tx[tx<total_min] = total_min
            tx[tx>total_max] = total_max
            tx = (tx-total_min)/(total_max-total_min)
            
            x1[x1<bone_min] = bone_min
            x1[x1>bone_max] = bone_max
            x1 = (x1-bone_min)/(bone_max-bone_min)
            x2[x2<soft_min] = soft_min
            x2[x2>soft_max] = soft_max
            x2 = (x2-soft_min)/(soft_max-soft_min)
            x3[x3<brain_min] = brain_min
            x3[x3>brain_max] = brain_max 
            x3 = (x3-brain_min)/(brain_max-brain_min)
            x4[x4 < stroke_min] = stroke_min
            x4[x4 > stroke_max] = stroke_max
            x4 = (x4 - stroke_min)/(stroke_max - stroke_min)
            tx = randintensity(tx)
            tx = randcontrast(tx)
            
            x2 = randintensity(x2)
            x2 = randcontrast(x2)
            tx *= 2
            x2 *= 2
            tx -= 1
            x2 -= 1

            x = torch.cat((tx, x1, x2, x3, x4), 1)
            
            with torch.cuda.amp.autocast():
                logit_map = model(x)
                loss = loss_function(logit_map, y)
            
            scaler.scale(loss).backward()
            epoch_loss += loss.item()
            scaler.unscale_(optimizer)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            epoch_iterator.set_description(f"Training ({global_step} / {max_iterations} Steps) (loss={loss:2.5f})")
            if (global_step % eval_num == 0 and global_step != 0) or global_step == max_iterations:
                epoch_iterator_val = tqdm(val_loader, desc="Validate (X / X Steps) (dice=X.X)", dynamic_ncols=True)
                dice_val = validation(epoch_iterator_val)
                epoch_loss /= step
                epoch_loss_values.append(epoch_loss)
                metric_values.append(dice_val)
                if dice_val > dice_val_best:
                    dice_val_best = dice_val
                    global_step_best = global_step
                    torch.save({
                        'global_step': global_step,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'scheduler_state_dict': scheduler.state_dict(),
                    }, os.path.join(model_savepath, "zone3_model_epoch_SGD_%d_%.3f.pth" %(global_step, float(dice_val))))
                    print(
                        "Model Was Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(dice_val_best, dice_val)
                    )
                else:
                    print(
                        "Model Was Not Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {}".format(
                            dice_val_best, dice_val
                        )
                    )
                print(optimizer.param_groups[0]['lr'])
                scheduler.step()
            global_step += 1
        # print((global_step+1) // (len(train_ds)) % t_0)
        # if (global_step+1) // (len(train_ds)) % t_0 == 0:
        #     print("donedone")
        #     scheduler.base_lrs[0] = scheduler.base_lrs[0] * (0.7)
        #     t_0 *= 2
        
        return global_step, dice_val_best, global_step_best, t_0
     
    max_iterations = 250000
    eval_num = 125
    post_label = AsDiscrete(to_onehot=OAR_nums_plus_one)
    post_pred = AsDiscrete(argmax=True, to_onehot=OAR_nums_plus_one)
    dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
    global_step = 0
    dice_val_best = 0.0
    global_step_best = 0
    epoch_loss_values = []
    metric_values = []
    t_0 = T_0
    while global_step < max_iterations:
        global_step, dice_val_best, global_step_best, t_0 = train(global_step, train_loader, dice_val_best, global_step_best, t_0=t_0)
        epoch_loss_npy = np.array(epoch_loss_values)
        np.save(os.path.join(loss_savepath, "%d_loss.npy" %(int(global_step))), epoch_loss_npy)
    # total_case_num = 24
    # model.load_state_dict(torch.load(os.path.join(root_dir, name_oar.lower() + "_model_fold0_0.pth")))
    # model.eval()
    # original_nib_path = root_dir + r"/imcroppedval"
    # original_nib_path_list = os.listdir(original_nib_path)
    return None

    # with torch.no_grad():
    #     for case_num in range(total_case_num):
    #         start = time.time()
    #         template_nib = nib.load(os.path.join(original_nib_path, original_nib_path_list[case_num]))

    #         img_name = os.path.split(val_ds[case_num]["image"].meta["filename_or_obj"])[1]
    #         img = val_ds[case_num]["image"]
    #         label = val_ds[case_num]["label"]
    #         val_inputs = torch.unsqueeze(img, 0).cuda()
    #         val_labels = torch.unsqueeze(label, 0).cuda()
    #         bone_min = -1000
    #         bone_max = 2000
    #         soft_min = -160
    #         soft_max = 350
    #         brain_min = -5
    #         brain_max = 65
    #         stroke_min = 15
    #         stroke_max = 45

    #         # box_start, box_end = FG_cropper.compute_bounding_box(val_inputs)
    #         tx =  torch.reshape(val_inputs[:, 0, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
    #         x1 =  torch.reshape(val_inputs[:, 0, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
    #         x2 =  torch.reshape(val_inputs[:, 0, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
    #         x3 =  torch.reshape(val_inputs[:, 0, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
    #         x4 =  torch.reshape(val_inputs[:, 0, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
    #         tx2 = torch.reshape(val_inputs[:, 1, :, :, :].clone().detach(), (val_inputs.shape[0], 1, val_inputs.shape[2], val_inputs.shape[3], val_inputs.shape[4]))
    #         x1[x1<bone_min] = bone_min
    #         x1[x1>bone_max] = bone_max
    #         x1 = (x1-bone_min)/(bone_max-bone_min)
    #         x2[x2<soft_min] = soft_min
    #         x2[x2>soft_max] = soft_max
    #         x2 = (x2-soft_min)/(soft_max-soft_min)
    #         x3[x3<brain_min] = brain_min
    #         x3[x3>brain_max] = brain_max 
    #         x3 = (x3-brain_min)/(brain_max-brain_min)
    #         x4[x4<stroke_min] = stroke_min
    #         x4[x4>stroke_max] = stroke_max
    #         x4 = (x4 - stroke_min)/(stroke_max - stroke_min)
    #         val_inputs = torch.cat((tx, x1, x2, x3, x4, tx2), 1)

    #         val_outputs = sliding_window_inference(val_inputs, spatial_size_xyz, 4, model, overlap=0.5)
    #         last_outputs = torch.argmax(val_outputs, dim=1).detach().cpu()[0].numpy()
    #         img_npy = img.cpu()[0].numpy()
    #         label_npy = label.cpu()[0].numpy()
    #         print("time taken : %f" %(time.time()- start))
    #         # nib.save(
    #                 # nib.Nifti1Image(img_npy.astype(np.uint8), np.ones((4, 4))), os.path.join(r"D:\!HaN_Challenge\HaN-Seg_NRRD\!output_dir\label_1to10_fold0", "val_ct_%02d" %(case_num+1))
    #             # )

    #         nib.save(
    #                 nib.Nifti1Image(label_npy.astype(np.uint8), template_nib.affine, template_nib.header), os.path.join(root_dir + "/result", "val_label_%02d" %(case_num+1))
    #             )
    #         nib.save(
    #                 nib.Nifti1Image(last_outputs.astype(np.uint8), template_nib.affine, template_nib.header), os.path.join(root_dir + "/result", "infered_label_%02d" %(case_num+1))
    #             )

In [4]:


json_name_list = [
                "dataset_f0_fold00.json", 
                "dataset_f0_fold01.json", 
                "dataset_f0_fold02.json", 
                "dataset_f0_fold03.json", 
                "dataset_f0_fold04.json"]
for json in json_name_list:
    oar_run_train(json)
    

Training (28480 / 250000 Steps) (loss=0.73594):   0%|          | 0/40 [00:00<?, ?it/s]

0.008972446262363005


Training (28501 / 250000 Steps) (loss=0.71397):  55%|█████▌    | 22/40 [00:11<00:33,  1.88s/it]

Model Was Not Saved ! Current Best Avg. Dice: 0.7203604578971863 Current Avg. Dice: 0.6620341539382935
0.008972446262363005


Training (X / X Steps) (loss=X.X):   0%|          | 0/40 [00:00<?, ?it/s]

...

0.00462476772039727


Training (143959 / 250000 Steps) (loss=0.75736): 100%|██████████| 40/40 [00:06<00:00,  5.99it/s]
Training (143960 / 250000 Steps) (loss=0.92654):   0%|          | 0/40 [00:00<?, ?it/s]

0.00462476772039727


Training (143980 / 250000 Steps) (loss=0.79784):  52%|█████▎    | 21/40 [00:03<00:03,  5.79it/s]


KeyboardInterrupt: 

fold 0 
arytenoid 0.64386
a_carotid_l 0.83728 (아마이것보단 높을것)
a_carotid_r 0.88483
bone_mandible 0.964496
oralcavity 0.924
cochlea_l 0.85533
optnr_1 0.77

fold 1 
arytenoid 0.45... 가우시안 노이즈 + 가우시안 블러링